http://knot.fit.vutbr.cz/corpproc/vertical_en.html

http://nlpserver2.inf.ufrgs.br/brwac-download/

In [ ]:
#! wget http://nlpserver2.inf.ufrgs.br/brwac-download/brwac.vert.gz
#! gzip -d brwac.vert.gz

--2021-09-11 17:41:41--  http://nlpserver2.inf.ufrgs.br/brwac-download/brwac.vert.gz
Resolving nlpserver2.inf.ufrgs.br (nlpserver2.inf.ufrgs.br)... 143.54.85.29
Connecting to nlpserver2.inf.ufrgs.br (nlpserver2.inf.ufrgs.br)|143.54.85.29|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7376518407 (6.9G) [application/octet-stream]
Saving to: ‘brwac.vert.gz’

brwac.vert.gz       100%[===================>]   6.87G  11.2MB/s    in 10m 29s 

2021-09-11 17:52:11 (11.2 MB/s) - ‘brwac.vert.gz’ saved [7376518407/7376518407]



In [1]:
import sys
import xml.sax
import subprocess
from urllib.parse import urlparse
import os
import shutil
from xml.etree.ElementTree import fromstring

In [2]:
remove_terms = ['blog', 'transparenciacapixaba', 'wordpress', 'politica', 'PCdoB', 'PSDB', 'podemos', 'PSOL', 'novo.org', 'redesp', 'redesustentabilidade', 'PSTU', 'PRTB', 'mdb', 'pdt', 'dem.org', 'ptb', 'psb', 'PSC', 'PSD', 'PMN', 'PTC', 'PSL', 'PMB', 'PCB', 'pt.org', 'pp.org' ]
remove_terms = [a.lower() for a in remove_terms]

print(f"Total de {len(remove_terms)} termos para descartar dominios")

Total de 27 termos para descartar dominios


In [3]:
contador_file = 0
 
brwac_path = '../Base/brwac.vert'
names_path = '../Base/names.tsv'

if os.path.exists("./data/"):
    shutil.rmtree("./data/")
os.mkdir("./data/")


with open(names_path, 'w') as fp_names:
    title = None
    url = None
    content_buffer = []
    for line in subprocess.Popen(['cat'], 
                                stdin = open(brwac_path), 
                                stdout = subprocess.PIPE).stdout:
        
        line = line.decode("UTF-8").replace("\n", "").replace("\t"," ")
        if line[0:4]=="<doc":
            if title!=None:
                if content_buffer[0]=="\n":
                    content_buffer.pop(0)

                is_ok = True
                for rem in remove_terms:
                    if rem in url:
                        is_ok = False
                        break

                if is_ok:
                    fp_names.write(f"{contador_file}\t{title}\t{url}\n")
                    content_buffer = ' '.join(content_buffer)
                    with open(f"./data/{contador_file}.txt", 'w') as fp:
                        fp.write(content_buffer)

                contador_file += 1
                content_buffer = []
                is_ok = True

            try:
                line = line.split("uri=\"")
                url = line[1][:-2]
                title = line[0].split("title=\"")[1][:-2]
            except Exception as e:
                raise e
        
        elif line=='<p>':
            content_buffer.append("\n")
        elif line=='<s>' or line=='</p>' or line=='</s>' or line=='<g/>' or line=='</doc>':
            pass
        else:
            content_buffer.append(line)

In [ ]:

lines = None
with open("./domains.txt", 'r') as fp:
    lines = fp.readlines()

print(len(lines))

domains_found = {}

for line in lines:
    domain = urlparse(line).netloc
    try:
        domains_found[domain] += 1
    except:
        domains_found[domain] = 1

domains_counter = []
for key in domains_found.keys():
    domains_counter.append({'domain': key, 'total': domains_found[key]})

domains_counter.sort(key=lambda x: x['total'], reverse=True)

In [ ]:

clean_domains = domains_counter.copy()


total_pages_b = sum([d['total'] for d in clean_domains])
total_domains_b = len(clean_domains)
print(f"Total domains: {total_domains_b} having {total_pages_b} pages")

for rem in remove_terms:
    removidos = [d['domain'] for d in clean_domains if rem in d['domain']]
    print("=>", rem, len(removidos), removidos)
    clean_domains = [d for d in clean_domains if rem not in d['domain']]


print("After")
total_pages_a = sum([d['total'] for d in clean_domains])
total_domains_a = len(clean_domains)
print(f"Total domains: {total_domains_a} having {total_pages_a} pages")

print(f"Removed {total_domains_b-total_domains_a} domains and {total_pages_b-total_pages_a} pages")

# clean_domains[0:100] # contem os dominios liberados

Total domains: 120989 having 3530796 pages
=> blog 23856 ['www.blogdochaguinhas.com.br', 'blog.opovo.com.br', 'eapnimprensa.blogspot.com.br', 'blogs.diariodonordeste.com.br', 'zelmar.blogspot.com.br', 'blogdomeireles.com.br', 'montedo.blogspot.com.br', 'ambicanos.blogspot.com.br', 'poncheverde.blogspot.com.br', 'juliosevero.blogspot.com.br', 'libertanimalteu04.blogspot.com.br', 'navblog.uol.com.br', 'desenvolturasedesacatos.blogspot.com.br', 'brazilianspace.blogspot.com.br', 'escrevalolaescreva.blogspot.com.br', 'comentandoanoticia.blogspot.com.br', 'blogs.estadao.com.br', 'blogdeklau.blogspot.com.br', 'blogdofumanchuphb.blogspot.com.br', 'www.blogdosaba.com.br', 'blogs.ne10.uol.com.br', 'www.blogers.com.br', 'www.blogosdoisladosdamoeda.com.br', 'gecorp.blogspot.com.br', 'blogdobirner.virgula.uol.com.br', 'estrelaseplanetas-cesar.blogspot.com.br', 'oblogdoabelha.blogspot.com.br', 'claudiopaguiar.blogspot.com.br', 'historiaupf.blogspot.com.br', 'beleza.blog.br', 'bairrodooriente.blogspo

In [ ]:
#! zip -rq BrWac.zip ./data/ names.tsv